## Introduction.
Hamburg is the second-largest city in Germany and is known to be the greenest city in the Country, rich in recreational areas.\
Moving inside Hamburg to an area closer to my work, I was looking for a place to live in a neighborhood where I could also easily walk my dog and play basketball in the evenings. Searching for a new dwelling I realized that boroughs of the city distinguishes from each other in this aspect.\
In this project I would like to explore neighborhoods in Hamburg for availability of sports grounds and green areas. Such clustering and its visualisation can provide with an information about parts of the city with a lack of places to rest. This information can be used by management of the city to make Hamburg even more better place to live. Moreover, this report is useful for someone who is looking for a place in Hamburg for recreational purposes.


## Description of the data to be used in the analysis.
The data for the analysis will be combined from two data sets. The first one contains geographical coordinates for every postal code in Hamburg. The second one provides us with information about boroughs and neighborhoods corresponding to each postal code.\
Resulting data set will contain boroughs and neighborhoods, postal codes and related longitude and latitude. It will be used to explore the neighborhoods and segment them with the Foursquare API. With the k-means clustering algorithm we will cluster the neighbourhoods in Hamburg and visualise them for observations and analysis.

In [1]:
!pip install bs4

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1272 sha256=29b46d4daeec2311145b7d8e89bffb27aca969e01b3ffbc307893ba940e6ba8a
  Stored in directory: /tmp/wsuser/.cache/pip/wheels/0a/9e/ba/20e5bbc1afef3a491f0b3bb74d508f99403aabe76eda2167ca
Successfully built bs4


In [ ]:
import requests # library to handle requests

In [20]:
from bs4 import BeautifulSoup # this module helps in web scrapping.
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files


from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

print('Libraries imported.')

Libraries imported.


In [2]:
url='https://launix.de/launix/wp-content/uploads/2019/06/PLZ.csv'

In [3]:
headers = ["PostalCode","City","Longitude","Latitude"]

In [4]:
df = pd.read_csv(url, skiprows=3, names = headers, sep=';')

In [5]:
df_h=df[df['City']=='Hamburg'].reset_index(drop=True)

df_h.head()

,PostalCode,City,Longitude,Latitude
0,20095,Hamburg,10.001104,53.554158
1,20097,Hamburg,10.020163,53.549704
2,20099,Hamburg,10.011777,53.560801
3,20144,Hamburg,9.975633,53.577054
4,20146,Hamburg,9.978171,53.570563


In [6]:
!conda install -c anaconda xlrd --yes

Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: failed with repodata from current_repodata.json, will retry with next repodata source.
Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python-3.7-main

  added / updated specs:
    - xlrd


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    _libgcc_mutex-0.1          |             main           3 KB  anaconda
    _py-xgboost-mutex-2.0      |            cpu_0           9 KB  anaconda
    _pytorch_select-0.1        |            cpu_0           2 KB  anaconda
    absl-py-0.10.0             |           py37_0         169 KB  anaconda
    aiohttp-3.6.3              |   py37h7b6447c_0         626 KB  anaconda
    argon2-cffi-20.1.0         |   py37h7b6447c_1          49

In [7]:
column_names= ['1','2','3','4','5','6','7','8','Neighborhood', 'Borough',"PostalCode",'12']
url1='https://www.statistik-nord.de/fileadmin/Dokumente/Wahlen/Hamburg/Bezirksversammlungswahlen/2019/Vor_der_Wahl/2018-06-19Stra%C3%9FenHH_WahlkreisBVWahl_Stadtteil_PLZ.xlsx'

In [21]:
df_plz = pd.read_excel(url1,names = column_names)

In [9]:
#drop the first row
df_plz = df_plz.iloc[1:]

In [10]:
df_plz.head(3)

,1,2,3,4,5,6,7,8,Neighborhood,Borough,PostalCode,12
1,Aalheitengraben,1,NaN,3,NaN,1,57,"Lemsahl-Mellingstedt, Duvenstedt, Wohldorf-Ohl...",Volksdorf,Wandsbek,22359,A001
2,Aalheitengraben,4,NaN,14,NaN,2,57,"Lemsahl-Mellingstedt, Duvenstedt, Wohldorf-Ohl...",Volksdorf,Wandsbek,22359,A001
3,Aalkrautweg,1,NaN,35,a,1,55,"Wellingsbüttel, Sasel",Sasel,Wandsbek,22395,A002


In [11]:
df_test=df_plz.drop(['1','2','3','4','5','6','7','8','12'], axis=1)
df_test.head()

,Neighborhood,Borough,PostalCode
1,Volksdorf,Wandsbek,22359
2,Volksdorf,Wandsbek,22359
3,Sasel,Wandsbek,22395
4,Sasel,Wandsbek,22395
5,Sasel,Wandsbek,22393


In [12]:
df_hamburg = pd.merge(df_h, df_test, on='PostalCode',how='inner')
df_hamburg.head()

,PostalCode,City,Longitude,Latitude,Neighborhood,Borough
0,20095,Hamburg,10.001104,53.554158,Hamburg-Altstadt,Hamburg-Mitte
1,20095,Hamburg,10.001104,53.554158,Hamburg-Altstadt,Hamburg-Mitte
2,20095,Hamburg,10.001104,53.554158,Hammerbrook,Hamburg-Mitte
3,20095,Hamburg,10.001104,53.554158,Hamburg-Altstadt,Hamburg-Mitte
4,20095,Hamburg,10.001104,53.554158,Hamburg-Altstadt,Hamburg-Mitte


In [13]:
df_hamburg.shape

(15491, 6)

In [14]:
#drop duplicates
df_hamburg.sort_values("Neighborhood", inplace = True)
df_hamburg.drop_duplicates(subset="Neighborhood", keep='first', inplace=True)

In [22]:
df_hamburg.reset_index(drop=True)

,PostalCode,City,Longitude,Latitude,Neighborhood,Borough
0,21037,Hamburg,10.144101,53.450581,Allermöhe,Bergedorf
1,22297,Hamburg,10.012196,53.611496,Alsterdorf,Hamburg-Nord
2,21039,Hamburg,10.261889,53.455322,Altengamme,Bergedorf
3,21129,Hamburg,9.848822,53.524423,Altenwerder,Harburg
4,22767,Hamburg,9.947594,53.553175,Altona-Altstadt,Altona
5,22765,Hamburg,9.932502,53.557255,Altona-Nord,Altona
6,22607,Hamburg,9.879987,53.571865,Bahrenfeld,Altona
7,22305,Hamburg,10.049692,53.591714,Barmbek-Nord,Hamburg-Nord
8,22085,Hamburg,10.015176,53.574655,Barmbek-Süd,Hamburg-Nord
9,21035,Hamburg,10.150629,53.494515,Bergedorf,Bergedorf


# define Foursquare Credentials and Version

In [16]:
CLIENT_ID = 'PRGJP2KTHQJW2GSA1O4BJC1UTQCBLQ5SPTGOAH4BN3TXAUAF' # your Foursquare ID
CLIENT_SECRET = 'A4V0FBT01TBMDP4E24R52W2K3SYQNZF023AT1ZGV0D2MCZ3X' # your Foursquare Secret

VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: PRGJP2KTHQJW2GSA1O4BJC1UTQCBLQ5SPTGOAH4BN3TXAUAF
CLIENT_SECRET:A4V0FBT01TBMDP4E24R52W2K3SYQNZF023AT1ZGV0D2MCZ3X


In [23]:
!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python-3.7-main

  added / updated specs:
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    _libgcc_mutex-0.1          |      conda_forge           3 KB  conda-forge
    _openmp_mutex-4.5          |           1_llvm           5 KB  conda-forge
    _py-xgboost-mutex-2.0      |            cpu_0           8 KB  conda-forge
    _pytorch_select-0.2        |            gpu_0           2 KB
    absl-py-0.12.0             |     pyhd8ed1ab_0          96 KB  conda-forge
    aiohttp-3.7.4              |   py37h5e8e339_0         632 KB  conda-forge
    altair-4.1.0               |             py_1         614 KB  conda-forge
   

In [24]:
!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

Solving environment: | 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - conda-forge/linux-64::pytorch==1.8.0=cpu_py37hafa7651_0
  - defaults/noarch::ibm-wsrt-py37main-keep==0.0.0=2020
  - defaults/noarch::ibm-wsrt-py37main-main==custom=2020
done

## Package Plan ##

  environment location: /opt/conda/envs/Python-3.7-main

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-2.1.0                |     pyhd3deb0d_0          64 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          98 KB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy   

In [25]:
address = 'Hamburg'

geolocator = Nominatim(user_agent="h_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Hamburg are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Hamburg are 53.550341, 10.000654.


In [27]:
# create map of Manhattan using latitude and longitude values
map_hamburg = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(df_hamburg['Latitude'], df_hamburg['Longitude'], df_hamburg['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_hamburg)  
    
map_hamburg